## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-23-21-02-36 +0000,nypost,Jersey City wasted $20m on museum which has n...,https://nypost.com/2026/02/23/us-news/jersey-c...
1,2026-02-23-21-02-05 +0000,nyt,Training for New ICE Agents Is ‘Deficient’ and...,https://www.nytimes.com/2026/02/23/us/politics...
2,2026-02-23-20-57-44 +0000,cbc,Federal government seeking input to develop me...,https://www.cbc.ca/news/politics/mens-health-f...
3,2026-02-23-20-57-36 +0000,startribune,Walz appoints Shireen Gandhi to lead embattled...,https://www.startribune.com/shireen-gandhi-min...
4,2026-02-23-20-57-31 +0000,nyt,Live Updates: Mexico Remains on Edge After Kil...,https://www.nytimes.com/live/2026/02/23/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2512/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,45
39,cartel,22
35,mexico,20
38,killing,14
418,killed,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
276,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...,135
268,2026-02-23-02-58-30 +0000,nypost,How brutal assasin ‘El Mencho’ became Mexico’s...,https://nypost.com/2026/02/22/world-news/who-i...,128
133,2026-02-23-17-36-20 +0000,nyt,Drug Lord ‘El Mencho’ Was Killed in Mexico. Wh...,https://www.nytimes.com/2026/02/23/world/ameri...,116
183,2026-02-23-14-17-01 +0000,nypost,Expert’s chilling warning after killing of car...,https://nypost.com/2026/02/23/world-news/exper...,112
8,2026-02-23-20-56-14 +0000,nypost,Inside the deadly raid to take out Mexico’s po...,https://nypost.com/2026/02/23/world-news/insid...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
276,135,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...
268,128,2026-02-23-02-58-30 +0000,nypost,How brutal assasin ‘El Mencho’ became Mexico’s...,https://nypost.com/2026/02/22/world-news/who-i...
135,85,2026-02-23-17-28-59 +0000,nypost,Ex-British Ambassador to US Peter Mandelson ar...,https://nypost.com/2026/02/23/world-news/ex-br...
298,53,2026-02-22-22-13-39 +0000,nypost,Panicked passengers run for lives at Guadalaja...,https://nypost.com/2026/02/22/world-news/panic...
169,43,2026-02-23-15-34-59 +0000,latimes,Nick Reiner pleads not guilty to murder in kil...,https://www.latimes.com/california/story/2026-...
200,39,2026-02-23-12-08-15 +0000,nypost,Trump invites gold-winning US men’s Olympic ho...,https://nypost.com/2026/02/23/us-news/trump-in...
244,36,2026-02-23-08-57-16 +0000,wsj,Stock Market Today: Dow Falls More Than 800 Po...,https://www.wsj.com/livecoverage/stock-market-...
16,36,2026-02-23-20-45-00 +0000,cbc,U.S. tells some of its diplomats and their fam...,https://www.cbc.ca/news/world/us-diplomats-tol...
42,34,2026-02-23-20-15-30 +0000,nypost,Netflix’s Sarandos shrugs off Trump’s demand t...,https://nypost.com/2026/02/23/business/netflix...
6,33,2026-02-23-20-56-30 +0000,nypost,NYC blizzard officially one of the top 10 bigg...,https://nypost.com/2026/02/23/us-news/nyc-bliz...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
